In [2]:
from mongoengine import *
import os
import pandas as pd
from pandas import Series,DataFrame
import datetime

## 1 读取文件

### 读取方式1：使用pandas

In [7]:
targetfilename=r"5622.txt"
# 家中mac
# targetpath=r"/Users/casablanca/03project/typhoonSearchSys/demo_data"
# 单位mac
targetpath=r"/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data"
fullname=os.path.join(targetpath,targetfilename)

In [8]:
fullname

'/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data/5622.txt'

不使用pandas的read_table

In [9]:
with open(fullname,'rb') as f:
    data=pd.read_table(f,sep='\n',encoding='utf-8',header=None,infer_datetime_format=False)
    print('读取成功')

读取成功


In [10]:
data.head(8)

,0
0,1
1,S.N. 122 5622 9 4 QI...
2,0 1 2 3 4 5 6 7 8 9 ...
3,+ ...
4,9 159 156 135 107 90 93 110 123 145 141 ...
5,4 121 109 90 71 58 53 59 73 87 96 ...
6,30 38 47 45 36 32 40 51 50 58 45 ...
7,9 156 159 142 126 106 100 98 -- -- -- ...


## 2 按行进行切分

In [11]:
len(data)

329

In [12]:
mark_list=[]
mark_list=[3,
 21,
 37,
 54,
 72,
 89,
 107,
 124,
 142,
 159,
 176,
 193,
 210,
 227,
 244,
 262,
 279,
 297,
 314]

In [129]:
mark_list[2]

37

In [132]:
data.iloc[37-3:40]

,0
34,MAX ...
35,S.N. 37950 5622 9 4 Y...
36,0 1 2 3 4 5 6 7 8 9 ...
37,+ ...
38,9 422 407 386 364 336 313 293 282 317 337 ...
39,4 399 376 351 329 308 289 282 294 315 332 ...


In [11]:
step_interval=

SyntaxError: invalid syntax (<ipython-input-11-81ea31d0963b>, line 1)

In [13]:
data.iloc[21-3][0]

'1'

In [14]:
type(data[:37][0])

pandas.core.series.Series

In [15]:
type(data[:37])

pandas.core.frame.DataFrame

- 注意data[:37]本身是一个dataframe，是一个columns只有一列的df

In [16]:
data[:37].head()

,0
0,1
1,S.N. 122 5622 9 4 QI...
2,0 1 2 3 4 5 6 7 8 9 ...
3,+ ...
4,9 159 156 135 107 90 93 110 123 145 141 ...


In [135]:
data[:45][0]

0                                                     1
1             S.N.   122      5622      9   4     QI...
2             0   1   2   3   4   5   6   7   8   9 ...
3     +                                             ...
4       9   159 156 135 107  90  93 110 123 145 141 ...
5       4   121 109  90  71  58  53  59  73  87  96 ...
6      30    38  47  45  36  32  40  51  50  58  45 ...
7       9   156 159 142 126 106 100  98  --  --  -- ...
8       5   130 129 115  95  77  67  65  74  89 101 ...
9      HM    26  30  27  31  29  33  33  --  --  -- ...
10                                                  ...
11      9   188 193 187 165 139 122 111 110 115 111 ...
12      6   131 137 135 119  99  84  78  81  93 108 ...
13     02    57  56  52  46  40  38  33  29  22   3 ...
14                                                  ...
15      9    --  --  --  --  --  --  --  --  --  -- ...
16      7   127 132 140 135 118 100  92  91  98 112 ...
17     03    --  --  --  --  --  --  --  --  -- 

找到截取的数据  
- 本次标记位到 下次标记位_index-2 为 **一个测站的该台风过程中的观测数据**

In [18]:
data[21-3:37-2][0]

18                                                    1
19            S.N.     0      5622      9   2     LI...
20            0   1   2   3   4   5   6   7   8   9 ...
21    +                                             ...
22      9   379 346 366 254 201 151 109  97 117 173 ...
23      2   395 370 329 281 231 183 146 133 150 199 ...
24     28   -16 -24  37 -27 -30 -32 -37 -36 -33 -26 ...
25      9   403 405 384 342 289 226 172 121 103 117 ...
26      3   398 406 387 346 294 238 181 134 110 122 ...
27     29     5  -1  -3  -4  -5 -12  -9 -13  -7  -5 ...
28      9   373 419 425 429 393 334 264 196 135 107 ...
29      4   360 405 421 401 355 298 234 171 118  92 ...
30     30    13  14   4  28  38  36  30  25  17  15 ...
31      9   352 433 465 460 429 381 322 253 196 145 ...
32      5   295 370 422 437 411 359 296 227 162 110 ...
33     HM    57  63  43  23  18  22  26  26  34  35 ...
34              MAX                                 ...
Name: 0, dtype: object

具体切分步骤：  
1. 找到当前标记位  
2. 计算当前标记位距离下一个标记位-2的长度  
3. 一行一行读取  
4. 以3为步长读取（注意需要加入一个是否含max的判断），并叠加当前时间

### S1 引入的model

In [19]:
class Extremum(EmbeddedDocument):
    '''
        极值
    '''
    occurredTime = DateTimeField()
    val = IntField()


class TideData(EmbeddedDocument):
    '''
        测站数据
    '''
    # 极大值的24小时观测数组
    forecast_arr = ListField(IntField())
    # 极小值得24小时观测数组
    realdata_arr = ListField(IntField())
    # 目标日期（年-月-日）
    #     targetdate=DateTimeField()
    targetdate = DateField()
    heigh_heigh_tide = EmbeddedDocumentField(Extremum)
    heigh_low_tide = EmbeddedDocumentField(Extremum)
    low_heigh_tide = EmbeddedDocumentField(Extremum)
    low_low_tide = EmbeddedDocumentField(Extremum)


class StationTideData(Document):
    '''
        测站潮位数据
    '''
    # 测站代码
    code = StringField(max_length=10)
    #     起始时间
    startdate = DateTimeField()
    #     测站名称
    stationname = StringField(max_length=10)
    #     gesjon数据
    point = PointField()
    #     平均海平面
    lev = IntField()
    #     警戒潮位
    jw = IntField()
    #     潮汐调和常数
    harmonicconstant = StringField()
    #     潮位数据
    realtidedata = ListField(EmbeddedDocumentField(TideData))

### S2 引入的之前定义好的方法

In [54]:
    def convert2StationBaseModel(ser: Series = None, **kwargs):
        '''
            根据传入的series，根据指定位置进行截取
        '''
        # 获取年份
        year = int(kwargs.get('year'))
        print(f'{ser[9][:-2]}.{ser[9][-2:]}|{ser[7][:-2]}.{ser[7][-2:]}')
        latlon = [float(f"{ser[9][:-2]}.{ser[9][-2:]}"), float(f"{ser[7][:-2]}.{ser[7][-2:]}")]
        code = ser[2]
        # 起始时间
        startdate = datetime.date(year, int(ser[3]), int(ser[4]))
        stationname = ser[5]
        # if stationname == 'SHACHENG':
        #     print(kwargs.get('index'))
        # 平均海平面
        lev = int(ser[11])
        # 警戒潮位
        jw = int(ser[13])
        # 潮汐调和常数
        harmonicconstant = ser[14]
        point = [latlon[1], latlon[0]]
        stationtidedata = StationTideData(code=code,
                                          startdate=startdate,
                                          stationname=stationname,
                                          lev=lev,
                                          jw=jw,
                                          harmonicconstant=harmonicconstant,
                                          point=point)
        return stationtidedata

In [55]:
    # 判断是否为下一个时间时间的起始位置
    def checkIsNextDateDataPoint(ser:Series):
        '''
            判断是否为日期的标志点
            传入一行进来
        '''

        if len(ser.split()) == 1:
            if ser.split()[0] in ["MAX", "MIN"]:
                return True
        else:
            return False

In [56]:
mark_list[:2][0]

3

In [57]:
checkpoint_arr=mark_list[:2]
checkpoint_arr[0+1]-1
for index,val in enumerate(range(0,20)):
    print(f'index:{index}|valu:{val}')
# for index,val in range(0,checkpoint_arr[0+1]-1):
#     print(f'index:{index}|valu:{val}')

index:0|valu:0
index:1|valu:1
index:2|valu:2
index:3|valu:3
index:4|valu:4
index:5|valu:5
index:6|valu:6
index:7|valu:7
index:8|valu:8
index:9|valu:9
index:10|valu:10
index:11|valu:11
index:12|valu:12
index:13|valu:13
index:14|valu:14
index:15|valu:15
index:16|valu:16
index:17|valu:17
index:18|valu:18
index:19|valu:19


In [111]:
    def splitData(**kwargs):
        '''
            TODO [*] 根据df按行进行拆分
        :param df:
        :return:
        '''

        # 1 获取截取的数组
        checkpoint_arr=mark_list[:2]
        index_columns=0
        year=kwargs.get('year')
        # 2 循环进行数据截取
        for index_checkpoint,val_checkpoint in enumerate(mark_list):
            print(f'index:{index_checkpoint}|val:{val_checkpoint}')
            # S 基本信息所在的行
            baseinfo_ser=Series(data.iloc[val_checkpoint-2][0].split())
            print(baseinfo_ser)
            # 从当前标志位开始到下一个标志位按行进行检索，并读取实时数据
            for index,val in enumerate(range(0,checkpoint_arr[index_checkpoint+1]-val_checkpoint)):
                print(f'val_checkpoint-2={val_checkpoint-2}')
                
                # 2.1 先读取基本信息
                base_model=convert2StationBaseModel(baseinfo_ser,year=year)
                # 获取当前时间
                temp_date=base_model.startdate

                temp_ser = data.iloc[val_checkpoint + index][0]

                # 判断是否为下一个时间节点标记位
                if index_columns<3:
                    index_columns = index_columns + 1
                    # 当前日期加1
                    temp_date=temp_date+datetime.timedelta(days=1)
                    print(temp_date)
                    if checkIsNextDateDataPoint(temp_ser) == False:
                        print(f'找到max或min，此处位置:{index}')
    #                     print(self.convert2RealData(data[val_checkpoint]))
                    else:
                        index_columns = 0
        
#         for index_checkpoint,val_checkpoint in enumerate(checkpoint_arr):

#             # 从当前标志位开始到下一个标志位按行进行检索，并读取实时数据
#             for index,val in range(0,checkpoint_arr[index_checkpoint+1]-val_checkpoint):
#                 # S 基本信息所在的行
#                 baseinfo_ser=data.iloc[index_checkpoint-2][0]
#                 # 2.1 先读取基本信息
#                 base_model=self.convert2StationBaseModel(baseinfo_ser)
#                 # 获取当前时间
#                 temp_date=base_model.startdate

#                 temp_ser = data.iloc[val_checkpoint + index][0]

#                 # 判断是否为下一个时间节点标记位
#                 if index_columns<3:
#                     index_columns = index_columns + 1
#                     # 当前日期加1
#                     temp_date=temp_date+datetime.timedelta(days=1)
#                     print(temp_date)
#                     if self.checkIsNextDateDataPoint(temp_ser) == False:
#     #                     print(self.convert2RealData(data[val_checkpoint]))
#                     else:
#                         index_columns = 0
# #                         break
            # for index in range(3):


        pass

In [112]:
splitData(year=1958)

index:0|val:3
0           S.N.
1            122
2           5622
3              9
4              4
5       QINHUANG
6            Lon
7          11937
8            Lat
9           3955
10           Lev
11            89
12            JW
13           220
14    H.G.Y.1975
dtype: object
val_checkpoint-2=1
39.55|119.37
1958-09-05
找到max或min，此处位置:0
val_checkpoint-2=1
39.55|119.37
1958-09-05
找到max或min，此处位置:1
val_checkpoint-2=1
39.55|119.37
1958-09-05
找到max或min，此处位置:2
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
val_checkpoint-2=1
39.55|119.37
index:1|val:21
0           S.N.
1              0
2       

IndexError: list index out of range

In [86]:
# ser=Series(list(map(int,data.iloc[-2][0].split())))
ser=Series(data.iloc[1][0].split())
ser

0           S.N.
1            122
2           5622
3              9
4              4
5       QINHUANG
6            Lon
7          11937
8            Lat
9           3955
10           Lev
11            89
12            JW
13           220
14    H.G.Y.1975
dtype: object

In [90]:
ser[9][:-2]

'39'

In [89]:
ser[9]

'3955'

In [101]:
# S 基本信息所在的行
baseinfo_ser=Series(data.iloc[1][0].split())
baseinfo_ser

0           S.N.
1            122
2           5622
3              9
4              4
5       QINHUANG
6            Lon
7          11937
8            Lat
9           3955
10           Lev
11            89
12            JW
13           220
14    H.G.Y.1975
dtype: object

In [102]:
[float(f"{baseinfo_ser[9][:-2]}.{baseinfo_ser[9][-2:]}"), float(f"{baseinfo_ser[7][:-2]}.{baseinfo_ser[7][-2:]}")]

[39.55, 119.37]

In [98]:
baseinfo_ser[9][:-2]

''

In [103]:
# 2.1 先读取基本信息
base_model=convert2StationBaseModel(baseinfo_ser,year=1958)

39.55|119.37


In [104]:
base_model

<StationTideData: StationTideData object>

In [115]:
# data

In [118]:
for i in range(1,3):
    print(i)

1
2


In [120]:
len(data.iloc[34])

1

In [121]:
data.iloc[34][0]

'          MAX                                                                                                                                 '

In [124]:
ser_temp=data.iloc[34][0].split()

In [128]:
ser_temp[0]

'MAX'

In [125]:
len(ser_temp)

1

In [126]:
ser_temp in ["MAX", "MIN"]

False

In [133]:
# 向数组中加入空值
list=[]
list.append(None)

In [134]:
list

[None]